In [1]:
import torch.nn as nn
import torch

In [3]:
class DeepFM(nn.Module):
    def __init__(self,features_size,embedding_size):
        super(DeepFM,self).__init__()
        
        self.features_size = features_size
        self.field_size = len(features_size)
        self.embedding_size = embedding_size
        
        ## FM first order embedding
        self.first_order_embeddings = nn.ModuleList([nn.Embedding(feature_size,1) for feature_size in features_size])
        ## FM second order embedding
        self.second_order_embeddings = nn.ModuleList([nn.Embedding(feature_size,self.embedding_size) for feature_size in features_size])
        ## Deep part
        self.dense_dim = self.field_size * self.embedding_size
        self.fc_dim = 500
        
        
        self.fc_layer1 = nn.Sequential(
                nn.Linear(dense_dim,fc_dim),
                nn.BatchNorm1d(fc_dim),
                nn.LeakyReLU(0.2, inplace=True)     
                )
 
        self.fc_layer2 = nn.Sequential(
                nn.Linear(fc_dim,fc_dim),
                nn.BatchNorm1d(fc_dim), 
                nn.LeakyReLU(0.2, inplace=True)     
                )
        
    
    def forward(self,xi,vi):
        ## input shape: bacth_size * field_size
        
        first_order_embeddings = self.first_order_embeddings
        second_order_embeddings = self.second_order_embeddings
        field_size = self.field_size
        ## FM first order part
        first_order_output_tmp1 = [ (first_order_embeddings[i](xi[:,i]).t()*vi[:,i]).t() for i in range(field_size)]
        first_order_output_tmp2 = torch.cat(first_order_output_tmp1,axis=1)
        first_order_output = torch.sum(first_order_output_tmp2,axis=1)
        
        ## FM second order part
        second_order_output_tmp1 = [(second_order_embeddings[i](xi[:,i]).t()*vi[:,i]).t() for i in range(field_size) ]
        ## 计算第一个二阶项, 固定 field不变
        sum_sec_order_output  = sum(second_order_output_tmp1)
        squared_sec_order_output = sum_sec_order_output*sum_sec_order_output
        sum_squared_sec_order_output = torch.sum(squared_sec_order_output,axis=1)/2
        ## 计算第二个二阶项
        squared_sec_list = [item*item for item in second_order_output_tmp1]
        sum_squared_sec = sum(squared_sec_list)
        sum_sum_squared_sec = torch.sum(sum_squared_sec,axis=1)/2
        ## finally
        second_order_output = sum_squared_sec_order_output-sum_sum_squared_sec
        
        ## Deep part
        dense_input = torch.cat(second_order_output_tmp1,axis=1)
        print(dense_input.shape)
        fc1_output = self.fc_layer1(dense_input)
        fc2_output = self.fc_layer2(fc1_output)
        
        ## sum all together
        sum_all = first_order_output+second_order_output+ torch.sum(fc2_output,axis=1)
        return sum_all

In [9]:
## dataloader
import pickle
with open('./processed/deepfm_training.pickle','rb') as fh: 
    total_data = pickle.load(fh)

In [57]:
import numpy as np
xi =np.array(total_data['xi'],dtype=float)
vi = np.array(total_data['vi'],dtype=float)
label = np.array(total_data['label'],dtype=float)
## convert numpy.ndarray to torch tensor object
xi = torch.tensor(xi)
vi = torch.tensor(vi)
label=torch.tensor(label)


In [58]:
from torch.utils.data import TensorDataset,DataLoader

In [59]:
class FM_Dataset(TensorDataset):
    def __init__(self,xi,vi,y=None):
        super(FM_Dataset,self).__init__()
        self.xi=xi
        self.vi=vi
        self.y=y
    def __len__(self):
        return self.xi.shape[0]
    def __getitem__(self,index):
        if self.y is not None: 
            sample = {'xi':self.xi[index],'vi':self.vi[index],'y':self.y[index]}
        else:
            sample = {'xi':self.xi[index],'vi':self.vi[index]}
        return sample
        

In [60]:
dataset = FM_Dataset(xi,vi,label)

In [63]:
batch_size=100
dataloader = DataLoader(dataset,shuffle=True,batch_size=batch_size)

In [65]:
## 定义损失函数
